# Hierarchical Indexing

In [61]:
import pandas as pd
import numpy as np

## A Multiply Indexed Series

### The bad way

In [62]:
index = [('California', 2000), ('California', 2010),
         ('New York', 2000), ('New York', 2010),
         ('Texas', 2000), ('Texas', 2010)]
populations = [33871648, 37253956,
               18976457, 19378102,
               20851820, 25145561]
pop = pd.Series(populations, index=index)
pop

(California, 2000)    33871648
(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
(Texas, 2010)         25145561
dtype: int64

In [63]:
pop[('California', 2010):('Texas', 2000)]

(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
dtype: int64

In [64]:
pop[[i for i in pop.index if i[1] == 2010]]

(California, 2010)    37253956
(New York, 2010)      19378102
(Texas, 2010)         25145561
dtype: int64

### The Better Way: Pandas MultiIndex

In [65]:
index = pd.MultiIndex.from_tuples(index)
index

MultiIndex([('California', 2000),
            ('California', 2010),
            (  'New York', 2000),
            (  'New York', 2010),
            (     'Texas', 2000),
            (     'Texas', 2010)],
           )

In [66]:
pop = pop.reindex(index)
pop

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [67]:
pop[:, 2010]

California    37253956
New York      19378102
Texas         25145561
dtype: int64

### MultiIndex as extra dimension

In [68]:
pop_df = pop.unstack()
pop_df

,2000,2010
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


In [69]:
pop_df.stack()

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [70]:
pop_df = pd.DataFrame({'total': pop,
                       'under18': [9267089, 9284094,
                                   4687374, 4318033,
                                   5906301, 6879014]})
pop_df

total  under18
California 2000  33871648  9267089
           2010  37253956  9284094
New York   2000  18976457  4687374
           2010  19378102  4318033
Texas      2000  20851820  5906301
           2010  25145561  6879014

In [71]:
f_u18 = pop_df['under18'] / pop_df['total']
f_u18.unstack()

,2000,2010
California,0.273594,0.249211
New York,0.247010,0.222831
Texas,0.283251,0.273568


## Methods of MultiIndex Creation

In [72]:
df = pd.DataFrame(np.random.rand(4, 2),
                  index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                  columns=['data1', 'data2'])
df

data1     data2
a 1  0.133023  0.219801
  2  0.096807  0.516711
b 1  0.069974  0.682063
  2  0.970286  0.835270

In [73]:
data = {('California', 2000): 33871648,
        ('California', 2010): 37253956,
        ('Texas', 2000): 20851820,
        ('Texas', 2010): 25145561,
        ('New York', 2000): 18976457,
        ('New York', 2010): 19378102}
pd.Series(data)

California  2000    33871648
            2010    37253956
Texas       2000    20851820
            2010    25145561
New York    2000    18976457
            2010    19378102
dtype: int64

### Explicit MultiIndex constructors

In [74]:
pd.MultiIndex.from_arrays([['a', 'a', 'b', 'b'], [1, 2, 1, 2]])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

In [75]:
pd.MultiIndex.from_tuples([('a', 1), ('a', 2), ('b', 1), ('b', 2)])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

In [76]:
pd.MultiIndex.from_product([['a', 'b'], [1, 2]])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

In [77]:
# 'labels' has been deprecated in newer versions of Pandas.
# pd.MultiIndex(levels=[['a', 'b'], [1, 2]],
#               labels=[[0, 0, 1, 1], [0, 1, 0, 1]])

# You should use 'codes' instead to define the index positions.
# 'codes' now replaces 'labels'. It refers to the position of the values in 'levels'.
index = pd.MultiIndex(levels=[['a', 'b'], [1, 2]], 
                      codes=[[0, 0, 1, 1], [0, 1, 0, 1]])

### MultiIndex level names

In [78]:
pop.index.names = ['state', 'year']
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

### MultiIndex for columns

In [79]:
# hierarchical indices and columns
index = pd.MultiIndex.from_product([[2013, 2014], [1, 2]],
                                   names=['year', 'visit'])
columns = pd.MultiIndex.from_product([['Bob', 'Guido', 'Sue'], ['HR', 'Temp']],
                                     names=['subject', 'type'])

# mock some data
data = np.round(np.random.randn(4, 6), 1)
data[:, ::2] *= 10
data += 37

# create the DataFrame
health_data = pd.DataFrame(data, index=index, columns=columns)
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      30.0  36.2  35.0  36.6  31.0  35.8
     2      61.0  38.5  14.0  38.5  26.0  36.3
2014 1      25.0  35.3  36.0  36.7  40.0  37.5
     2      38.0  36.2  39.0  37.2  22.0  36.2

In [80]:
health_data['Guido']

type          HR  Temp
year visit            
2013 1      35.0  36.6
     2      14.0  38.5
2014 1      36.0  36.7
     2      39.0  37.2

## Indexing and Slicing a MultiIndex

### Multiply indexed Series

In [81]:
pop['California', 2000]

33871648

In [82]:
pop['California']

year
2000    33871648
2010    37253956
dtype: int64

In [83]:
pop.loc['California':'New York']

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
dtype: int64

In [84]:
pop[:, 2000]

state
California    33871648
New York      18976457
Texas         20851820
dtype: int64

In [85]:
pop[pop > 22000000]

state       year
California  2000    33871648
            2010    37253956
Texas       2010    25145561
dtype: int64

In [86]:
pop[['California', 'Texas']]

state       year
California  2000    33871648
            2010    37253956
Texas       2000    20851820
            2010    25145561
dtype: int64

### Multiply indexed DataFrames

In [87]:
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      30.0  36.2  35.0  36.6  31.0  35.8
     2      61.0  38.5  14.0  38.5  26.0  36.3
2014 1      25.0  35.3  36.0  36.7  40.0  37.5
     2      38.0  36.2  39.0  37.2  22.0  36.2

In [88]:
health_data['Guido', 'HR']

year  visit
2013  1        35.0
      2        14.0
2014  1        36.0
      2        39.0
Name: (Guido, HR), dtype: float64

In [89]:
health_data.iloc[:2, :2]

subject      Bob      
type          HR  Temp
year visit            
2013 1      30.0  36.2
     2      61.0  38.5

In [90]:
health_data.loc[:, ('Bob', 'HR')]

year  visit
2013  1        30.0
      2        61.0
2014  1        25.0
      2        38.0
Name: (Bob, HR), dtype: float64

In [91]:
# This is incorrect because the parentheses cause a syntax error
# health_data.loc[(:, 1), (:, 'HR')]

In [92]:
idx = pd.IndexSlice
health_data.loc[idx[:, 1], idx[:, 'HR']]

,subject,Bob,Guido,Sue
,type,HR,HR,HR
year,visit,,,
2013,1,30.0,35.0,31.0
2014,1,25.0,36.0,40.0


## Rearranging Multi-Indices

### Sorted and unsorted indices

In [93]:
index = pd.MultiIndex.from_product([['a', 'c', 'b'], [1, 2]])
data = pd.Series(np.random.rand(6), index=index)
data.index.names = ['char', 'int']
data

char  int
a     1      0.064204
      2      0.881120
c     1      0.021475
      2      0.176275
b     1      0.825634
      2      0.745552
dtype: float64

In [94]:
data = data.sort_index()
data

char  int
a     1      0.064204
      2      0.881120
b     1      0.825634
      2      0.745552
c     1      0.021475
      2      0.176275
dtype: float64

In [95]:
data['a':'b']

char  int
a     1      0.064204
      2      0.881120
b     1      0.825634
      2      0.745552
dtype: float64

### Stacking and unstacking indices

In [96]:
pop.unstack(level=0)

state,California,New York,Texas
year,,,
2000,33871648,18976457,20851820
2010,37253956,19378102,25145561


In [97]:
pop.unstack(level=1)

year,2000,2010
state,,
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


In [98]:
pop.unstack().stack()

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

### Index setting and resetting

In [99]:
pop_flat = pop.reset_index(name='population')
pop_flat

,state,year,population
0,California,2000,33871648
1,California,2010,37253956
2,New York,2000,18976457
3,New York,2010,19378102
4,Texas,2000,20851820
5,Texas,2010,25145561


In [100]:
pop_flat.set_index(['state', 'year'])

population
state      year            
California 2000    33871648
           2010    37253956
New York   2000    18976457
           2010    19378102
Texas      2000    20851820
           2010    25145561

### Data Aggregations on Multi-Indices

In [101]:
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      30.0  36.2  35.0  36.6  31.0  35.8
     2      61.0  38.5  14.0  38.5  26.0  36.3
2014 1      25.0  35.3  36.0  36.7  40.0  37.5
     2      38.0  36.2  39.0  37.2  22.0  36.2

In [103]:
# The 'level' argument is no longer supported in the 'mean()' method.
# data_mean = health_data.mean(level='year')

# To calculate the mean for a specific level in a MultiIndex, we now use 'groupby(level=...)' instead.
# The 'groupby()' method allows us to group data by a specific level and then apply the 'mean()' function.
data_mean = health_data.groupby(level='year').mean()

data_mean

subject   Bob        Guido          Sue       
type       HR   Temp    HR   Temp    HR   Temp
year                                          
2013     45.5  37.35  24.5  37.55  28.5  36.05
2014     31.5  35.75  37.5  36.95  31.0  36.85

In [106]:
# data_mean.mean(axis=1, level='type')

data_mean = health_data.groupby(axis=1, level='type').mean()
data_mean

C:\Users\Andre\AppData\Local\Temp\ipykernel_17756\2792389061.py:3: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  data_mean = health_data.groupby(axis=1, level='type').mean()


type               HR       Temp
year visit                      
2013 1      32.000000  36.200000
     2      33.666667  37.766667
2014 1      33.666667  36.500000
     2      33.000000  36.533333

In [109]:
# The 'groupby()' method with axis=1 is deprecated.
# To group by a specific level in columns, we should transpose the DataFrame first using '.T',
# apply 'groupby()' on the level, and then compute the mean. After that, we transpose it back with '.T'.
data_mean = health_data.T.groupby(level='type').mean().T
data_mean

type               HR       Temp
year visit                      
2013 1      32.000000  36.200000
     2      33.666667  37.766667
2014 1      33.666667  36.500000
     2      33.000000  36.533333